In [1]:
import cv2
import numpy as np
import os

def detect_lanes(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Define a region of interest
    mask = np.zeros_like(edges)
    height, width = mask.shape
    polygon = np.array([[(0, height), (width, height), (width // 2, height // 2)]], np.int32)
    cv2.fillPoly(mask, polygon, 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Apply Hough Line Transform
    lines = cv2.HoughLinesP(masked_edges, 1, np.pi / 180, threshold=50, minLineLength=100, maxLineGap=50)

    # Draw the lines on the frame
    line_image = np.zeros_like(frame)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 5)

    # Combine the original frame with the lines
    result = cv2.addWeighted(frame, 0.8, line_image, 1, 0)

    return result

# Define a function to process a list of images
def process_images(image_paths):
    for image_path in image_paths:
        # Load the image
        frame = cv2.imread(image_path)

        # Detect lanes
        result = detect_lanes(frame)

        # Display the result
        cv2.imshow('Self-Driving Car', result)

        # Wait for a key press and then close the window
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# Define the directory containing your images
image_directory = 'car'  # Replace with the actual directory path

# Get a list of image file names in the directory
image_files = [f for f in os.listdir(image_directory) if f.endswith('.jpg') or f.endswith('.png')]

# Create a list of image paths
image_paths = [os.path.join(image_directory, f) for f in image_files]

# Process the images
process_images(image_paths)
